In [1]:
import config
import requests
import pandas as pd 
from bs4 import BeautifulSoup

In [2]:
years = list(range(config.startingYear, config.finalYear + 1))

types = ["totals", "advanced"]

In [3]:
url_start = "https://www.basketball-reference.com/leagues/NBA_{}_{}.html"

In [4]:
for t in types:
    for year in years:
        url = url_start.format(year, t)
        data = requests.get(url)

        with open("stats/{}_{}.html".format(t, year), "w+", encoding="utf-8") as f:
            f.write(data.text)


In [5]:
def fixTradedPlayers(df):
    for index, row in df.iterrows():
        if(row["Tm"] == "TOT"):
            totIndex = index
            dupePlayer = df.loc[df['Player']==row['Player']]
            for indexx, roww in dupePlayer.iterrows():
                team = roww.Tm # Ensures team is set to the most recent team the player played for
                if(indexx != totIndex):
                    df.drop([indexx], inplace=True) # Drop duplicate player rows, keeps total stats
            df.at[totIndex, "Tm"] = team
    return df

In [6]:
def addYearlyStats(df1, df2, year):
    
# Drop Unneeded Columns
    df2 = df2.drop(["Rk","Player","Pos","Age","Tm","G","MP","Unnamed: 19","Unnamed: 24"], axis=1)
# Add two DFs
    result = pd.concat([df1, df2], axis=1)
# Export to CSV
    result.to_csv("csvStats/stats_" + year + ".csv")

In [7]:
totalStats = {}
for t in types:
    if (t == "totals"): 
        idName = "totals_stats" 
    else: 
        idName = "advanced_stats" 
    for year in years:

        with open("stats/{}_{}.html".format(t, year), encoding="utf-8") as f:
            page = f.read()

        soup = BeautifulSoup(page, "html.parser") 
        totals = soup.find(id=idName)
        for tr in totals.find_all("tr", {'class':'thead'}): 
            tr.decompose()
        totalStats[t + "_" + str(year)] = pd.read_html(str(totals))[0]
        totalStats[t + "_" + str(year)] = fixTradedPlayers(totalStats[t + "_" + str(year)])

In [ ]:
for year in years:
    addYearlyStats(totalStats["totals_" + str(year)], totalStats["advanced_" + str(year)], str(year))